In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import os
from tqdm import tqdm

## Constructing a diet based on nutrient needs


### Diet diversity
Nutrients are contained in different quantities in different commodities; for example:
- roots and tubers contain a lot of vitamin A but a low amount of protein
- livestock and fish contain a lot of protein and vitamin B12
- some nutrients are generally harder to come by than others

Therefore a healthy diet requires food intake diversity.

Diversity is also important from an agricultural point of view; different types of crops or different species of a given crop will have less chance of all being wiped out by a single disease. 

### Malnutrition


The first cause of death by malnutrition is caused by a lack of protein. The most common mineral deficiency is iron deficiency.


Different regions have different diets and different deficiencies. In the USA for example, the nutrients that are under-consumed are fiber, calcium, vitamin D and potassium. However, in less-developped countries, there are a lot of iron and protein deficiencies.


### Plants nutrients
While plants contain a lot of fibers and carbohydrates, they have a poor balance of essential amino acids and have a low protein content. For example, cereals tend to be low in Lys (lysine) content, and legumes tend to be low in sulfur-rich amino acids (methionine, cysteine). It is important to note that manufacturing amino acids is costly, therefore manufacturing and distributing amino acids is not a feasible solution as of now. 

Another important point to consider is that some foods also contain anti-nutrients, allergens or toxins. A good example of this is phytate-containing plants. Phytate chelates iron, calcium and zinc which are essential nutrients. Therefore its positive impacts could be nefated by the fact that it impedes other nutrients from being properly incroporated by the human body.

### Meat nutrients
Although its overconsumption is an issue, meat is a major source of micro and macro nutrients, contains all essential amino acids and has overall high nutritional value. Its content in iron, zinc, vitamins B1 and B12, niacin, protein and vitamins A and D is high, with good amino acid digestibility (e.g. when compared to plants).
Food group:
- Energy
- Protein, total fat, fatty acids, cholesterol
- Micronutrients (iron, zinc, sodium, phosphorus, calcium, copper, magnesium)
- Vitamins (vitamin A, vitamin D, thiamin, riboflavin, niacin, vitamin B6, vitamin B12, vitamin E)


### Meat vs. plants:

Moreover, plants are a fairly poor substitute for some of the nutrients meat provides. For example, iron and zinc contents in plants are very low. Their absorption Is further reduced in phytate-containing plants. The main nutrients plants lack in high quantities are Vitamin A, Thiamin, Vitamin B12, zince, Calcium.

A diet relying more on plants rather than solely on meat is in accordance with most health councils and organizations. 
Vitamin B12 is mainly found in meat. Vitamin A is mainly found in butter, liver, and in carotenoids to some extent. Thiamin is found in meat and dairy, and cereals to a lesser extent. Calcium is mainly found in milk dairy, and cereals and vegetables to a much lesser extent. 


## Creating a nutrient list

These nutrients are deemed to be essential not only because of their paramount importance to a healthy diet but also because of the fact that there exists widespread deficiencies worldwide.

Some of these can be easily manufactured and distributed (e.g. iodine). We will only focus on a subset of nutrients, and look at protein content, fiber content, lipid content, vitamin content and mineral content of given foods in order to construct a "healthy" diet (meaning one that provides good nutrient intake).

- Proteins
- Carbohydrates
- Fibers
- Lipids
- Vitamins and minerals
    - Vitamins: A, B6, B9, B12, C, D, E
    - Minerals : Fe, Ca
    

# Nutrient value database
In parallel with the evaluation of the environmental impact of livestock and crops, we need to decide which food yield the best (or at least a good/healthy enough) nutritive value. 
To do this, after having decided on certain categories of nutrients (vitamins / fibers / etc...), we will check that the nutrients that we plan on using contain enough nutritive value using this database:

https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/food-surveys-research-group/docs/fndds-download-databases/

Files:

Ingredient Nutrient Values.xlsx

FNDDS Nutrient Values.xlsx

In [7]:
food_attr = pd.read_excel("../2015-2016_FNDDS_Nutrient_Values.xlsx", header = 1)

In [8]:
food_attr.head(1)

,Food code,Main food description,WWEIA Category code,WWEIA Category description,Energy (kcal),Protein (g),Carbohydrate (g),"Sugars, total (g)","Fiber, total dietary (g)",Total Fat (g),...,20:1 (g),22:1 (g),18:2 (g),18:3 (g),18:4 (g),20:4 (g),20:5 n-3 (g),22:5 n-3 (g),22:6 n-3 (g),Water (g)
0,11000000,"Milk, human",9602,Human milk,70,1.03,6.89,6.89,0.0,4.38,...,0.04,0.0,0.374,0.052,0.0,0.026,0.0,0.0,0.0,87.5


In [11]:
food_nutrient = pd.read_excel("../2015-2016 FNDDS At A Glance - Ingredient Nutrient Values.xlsx", header = 1 )

In [12]:
food_nutrient.head(1)

,Ingredient code,SR description,Nutrient code,Nutrient description,Nutrient value,Nutrient value source,SR 28 derivation code,SR 28 AddMod year
0,1001,"Butter, salted",203,Protein,0.85,SR28,NaN,1976.0


In [13]:
# food_nutrient["SR description"].unique()
food_nutrient = food_nutrient.drop(["Ingredient code", "Nutrient code", "Nutrient value source", "SR 28 derivation code", "SR 28 AddMod year"], axis = 1)
food_nutrient.columns = ["ingredient","nutrient","value"]
print(food_nutrient.ingredient.nunique(), food_nutrient.nutrient.nunique())

2730 65


In [14]:
nutrient_list = list(food_nutrient.nutrient.unique())
selected_nutrients = ["Protein","Total Fat","Carbohydrate","Energy","Water",\
                      "Fiber, total dietary","Iron",\
                      "Calcium", "Vitamin C","Vitamin B-6",\
                      "Vitamin B-12","Vitamin A, RAE","Vitamin E (alpha-tocopherol)"\
                      ,"Vitamin D (D2 + D3)"]
assert(set(selected_nutrients) - set(nutrient_list) == set())

### An example of how we could use this dataset:
First we would need to isolate certain food components based on environmental and agricultural considerations, Tand only then would we cross-check these components with the database here to make sure that they contain the necessary nutrients. 

In [15]:
# # food_nutrient[food_nutrient.nutrient == "Calcium"].sort_values("value", ascending = False).head()
df = pd.DataFrame(columns = ["name","value","item"])
for idx, val in enumerate(selected_nutrients):
    max_val= food_nutrient[food_nutrient.nutrient == val].value.max()
    max_item = food_nutrient[food_nutrient.nutrient ==val].value.idxmax()
    max_item = food_nutrient.iloc[max_item]
    df.loc[idx] = val, max_val, max_item
df
# here: issue with vitamin D value. 
# also: the max value is often a supplement / else...
# that's why it would be best used as a cross-checking tool
# since this is not our original database, we're not going to spend ages cleaning / arranging it. 

,name,value,item
0,Protein,88.32,ingredient Soy protein isolate nutrient ...
1,Total Fat,100.00,"ingredient Fat, beef tallow nutrient ..."
2,Carbohydrate,100.00,"ingredient Sweetener, herbal extract powder..."
3,Energy,902.00,"ingredient Fat, beef tallow nutrient ..."
4,Water,99.98,"ingredient Water, bottled, generic nutrient..."
5,"Fiber, total dietary",53.20,"ingredient Spices, curry powder nutrient ..."
6,Iron,123.60,"ingredient Spices, thyme, dried nutrient ..."
7,Calcium,7364.00,"ingredient Leavening agents, baking powder,..."
8,Vitamin C,2400.00,"ingredient Fruit-flavored drink, powder, wi..."
9,Vitamin B-6,12.00,"ingredient Cereals ready-to-eat, KELLOGG, K..."


In [23]:
food_nutrient[(food_nutrient.ingredient.str.contains("meat"))&(food_nutrient.ingredient.str.contains("raw"))].ingredient.unique()

array(['Chicken, broiler or fryers, breast, skinless, boneless, meat only, raw',
       'Chicken, broilers or fryers, thigh, meat and skin, raw',
       'Duck, domesticated, meat only, raw',
       'Turkey, whole, meat and skin, raw',
       'Turkey, whole, meat only, raw',
       'Chicken, broilers or fryers, breast, skinless, boneless, meat only, enhanced, raw',
       'Turkey, retail parts, enhanced, breast, meat only, raw',
       'Turkey, retail parts, enhanced, breast, meat and skin, raw',
       'Pork, fresh, variety meats and by-products, stomach, raw',
       'Nuts, coconut meat, raw',
       'Nuts, coconut milk, raw (liquid expressed from grated meat and water)',
       'Beef, variety meats and by-products, tripe, raw',
       'Game meat, deer, ground, raw',
       'Beef, ground, 95% lean meat / 5% fat, raw',
       'Beef, ground, 90% lean meat / 10% fat, raw',
       'Beef, ground, 85% lean meat / 15% fat, raw',
       'Beef, ground, 80% lean meat / 20% fat, raw',
       'Be

We will only take a look at beef and chicken for clarity purposes. We will also only consider raw meat as the cooking process alters the nutrient values.

In [75]:
chicken = food_nutrient[(food_nutrient.ingredient.str.contains("meat"))&(food_nutrient.ingredient.str.contains("raw"))&(food_nutrient.ingredient.str.contains("Chicken"))].reset_index()
beef = food_nutrient[(food_nutrient.ingredient.str.contains("meat"))&(food_nutrient.ingredient.str.contains("raw"))&(food_nutrient.ingredient.str.contains("Beef"))].reset_index()
chicken = chicken[chicken.nutrient.isin(selected_nutrients)].groupby(["nutrient"]).agg({"value":"mean"}).reset_index().rename({"value":"chicken"},axis=1)
beef = beef[beef.nutrient.isin(selected_nutrients)].groupby(["nutrient"]).agg({"value":"mean"}).reset_index().rename({"value":"beef"},axis=1)
chicken.merge(beef, on = "nutrient").sort_values("nutrient").style.background_gradient(cmap='Reds', low=0.2, high=0.8, axis =1 )


,nutrient,chicken,beef
0,Calcium,5.66667,24
1,Carbohydrate,0.0833333,0
2,Energy,149.667,193.333
3,"Fiber, total dietary",0,0
4,Iron,0.463333,1.83833
5,Protein,19.78,17.5
6,Total Fat,7.41,13.115
7,"Vitamin A, RAE",14.3333,3.33333
8,Vitamin B-12,0.34,2.04333
9,Vitamin B-6,0.639667,0.290833


We can see that chicken is lacking the Calcium, Iron and Vitamin B-12 when compared to beef. Therefore we would need to find a complementary substitute to replace beef with chicken.

In [93]:
food_nutrient[(food_nutrient.ingredient.str.contains("Milk, whole"))&(food_nutrient.nutrient.str.contains("Calcium"))]

,ingredient,nutrient,value
3390,"Milk, whole, 3.25% milkfat, with added vitamin D",Calcium,113.0
7030,"Milk, whole, 3.25% milkfat, without added vita...",Calcium,113.0


Milk is a good substitute for Calcium.

In [111]:
food_nutrient[(food_nutrient.ingredient.str.contains("Cereals ready-to-eat, granola, homemade"))&((food_nutrient.nutrient.str.contains("Iron")))]

,ingredient,nutrient,value
46031,"Cereals ready-to-eat, granola, homemade",Iron,3.95


Granola is made from different cereals and oats and is a good substitute for Iron deficiencies. It is harder to find Vitamin B-12 ingredients that could replace beef.

In [124]:
food_nutrient[(food_nutrient.nutrient == "Vitamin B-12")\
              &~(food_nutrient.ingredient.str.contains("Beef"))\
              &~(food_nutrient.ingredient.str.contains("Cereals"))]\
                .sort_values("value", ascending = False).head(15)

,ingredient,nutrient,value
109299,"Mollusks, octopus, common, cooked, moist heat",Vitamin B-12,36.00
41569,"Braunschweiger (a liver sausage), pork",Vitamin B-12,20.09
104879,"Fish, caviar, black and red, granular",Vitamin B-12,20.00
108844,"Mollusks, octopus, common, raw",Vitamin B-12,20.00
108974,"Mollusks, oyster, eastern, canned",Vitamin B-12,19.13
105854,"Fish, herring, Atlantic, kippered",Vitamin B-12,18.70
108649,"Mollusks, clam, mixed species, canned, drained...",Vitamin B-12,18.63
29934,"Chicken, liver, all classes, cooked, simmered",Vitamin B-12,16.85
29869,"Chicken, liver, all classes, raw",Vitamin B-12,16.58
31689,"Turkey, whole, giblets, cooked, simmered",Vitamin B-12,15.89


It will be hard to replace the Vitamin B-12 found in beef-based diets, but there are alternatives (fish, game meat, chicken liver)

# Fertilizer use


The increased use of fertilizer has a strong impact on the environment, however it does generally help agriculture. It is also deemed necessary in many regions, and many reports highlight the fact that in order to feed the whole population, the quantity of fertilizer use must increase. However, it needs to be regulated, and a lot of areas could use a lesser amount. A direct consequence of high fertilizer use is the formation of excess nitrogen areas e.g. in rivers or lakes.


Obviously, some crops and products require more fertilizer than others. An example of this is rice cultivation, which requires large amounts of area, water and 
fertilizers. Rice yields good energy, protein and carbohydrate nutritive value, but relatively low water, fat and fiber contents when compared to other staple foods such as maize, wheat or potatoes. Since rice is the most consumed food in the world, being the main diet ingredient of approximately half of the world's population. Therefore we want to investigate whether rice is a good agricultural choice both in terms of environmental impact and nutritive value at a worldwide scale.

# Agriculture Sectors Pollution


Greenhouse gases basically trap heat and contribute to global warming. GHGs are almost entirely antropogenic.
- C02: carbon dioxide. Deforestation, land clearing for agriculture and soil degradation impacts CO2 release in the air
- CH4: methane. Most agriculture practices, agricultural waster, biomass burning, and livestock impact CH4 release in the air.
- N20: Nitrous oxide. Fertilizer use is the main culrpit here, impacting N20 release in the air.

However, agricultural practices also have some positive impact on GHG release: some ecosystems offset part of CO2 emissions.

# References:

https://www.sciencedirect.com/science/article/pii/S2542519617300074

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2442550/

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3546624/

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6213018/

https://www.ncbi.nlm.nih.gov/pubmed/28532520

http://www.fao.org/3/T0207E04.htm

https://academic.oup.com/ajcn/article/78/3/660S/4690010

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5346408/

http://www.fao.org/3/ca6030en/ca6030en.pdf

https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3546624/

https://en.wikipedia.org/wiki/Rice#Nutrition

https://www.earth.columbia.edu/articles/view/3281

https://www.epa.gov/ghgemissions/sources-greenhouse-gas-emissions